In [1]:
# import neccessary libraries 
import numpy as np 
import matplotlib.pyplot as plt
import os 
from tkinter import Tcl

import ipywidgets as widgets

from PIL import Image 

from poreUtils import * 

In [2]:
# write a function to load volume from the path

def load_volume(volume_path):
    """Load the volume from the path provided.

    Args:
    volume_path (str): Path to the volume.

    Returns:
    vol (numpy array): The volume loaded from the path.

    Raises:
    Exception: If the path does not exist, raises an exception.
    """
    try:
        tiffs = os.listdir(volume_path)

        slices = Tcl().call('lsort', '-dict', tiffs)
        shape = Image.open(os.path.join(volume_path, slices[0])).size
        vol = np.empty(shape=(len(slices), shape[0], shape[1]), dtype=np.uint8)

        for i, fname in enumerate(slices):
            im = Image.open(os.path.join(volume_path, fname))
            imarray = np.array(im)
            vol[i, :, :] = imarray

        return vol

    except:
        raise Exception(f"{volume_path} : path does not exists")

In [3]:
def load_volume2(volume_path):
    """Load the volume from the path provided.

    Args:
    volume_path (str): Path to the volume.

    Returns:
    vol (numpy array): The volume loaded from the path.

    Raises:
    Exception: If the path does not exist, raises an exception.
    """
    try:
        tiffs = os.listdir(volume_path)

        slices = Tcl().call('lsort', '-dict', tiffs)
        shape = Image.open(os.path.join(volume_path, slices[0])).size
        vol = np.empty(shape=(len(slices), shape[0], shape[1]), dtype=np.uint8)

        # Temporary list to hold blank slices
        blank_slices = []

        for i, fname in enumerate(slices):
            im = Image.open(os.path.join(volume_path, fname))
            imarray = np.array(im)
            
            if np.all(imarray == 0):
                blank_slices.append(imarray)
            else:
                vol[i - len(blank_slices), :, :] = imarray

        # Append blank slices at the end
        if len(blank_slices) > 0:
            vol[-len(blank_slices):] = blank_slices

        return vol

    except:
        raise Exception(f"{volume_path} : path does not exist")



In [4]:
def interactive_visualize(arr, cmap='gray', **kwargs):
    """
    This function creates an interactive visualization of the input array in 2D or 3D.

    Parameters:
    arr (ndarray): The input array to be visualized.
    cmap (str): The colormap to use for the plot. Default is 'viridis'
    **kwargs: Additional keyword arguments to be passed to plt.imshow.

    Returns:
    None
    """
    
    if arr.ndim == 2:
        plt.imshow(arr, cmap=cmap, **kwargs)
        plt.show()
    elif arr.ndim == 3:
    
        slider = widgets.IntSlider(
            min=0, max=arr.shape[0]-1, step=1, value=np.rint(arr.shape[0] / 2))

        def visualize_morph(slice_number):
            """
            This function plots the 3D visualization of the input array.

            Parameters:
            slice_number (int): The slice number of the input array to be visualized.

            Returns:
            None
            """
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12,4))
            plt.subplots_adjust(hspace=0.1, wspace=0.1)

            ax1.imshow(arr[slice_number, :, :], cmap=cmap, **kwargs)
            ax2.imshow(arr[:, slice_number, :], cmap=cmap, **kwargs)
            ax2.set_xticks([])
            ax2.set_yticks([])
            ax3.imshow(arr[:, :, slice_number], cmap=cmap, **kwargs)
            ax3.set_xticks([])
            ax3.set_yticks([])
            plt.show()


        widgets.interact(visualize_morph, slice_number=slider)
    
    else:
        raise ValueError("Input array must be either 2D or 3D.")

In [5]:
scan_path = 'E:\\Data\\sam_data\\new\\MD_1264_A5_1_Z0.0mm_corr_phrt\\roi\\0-300x1550-1850x2250-2550'

In [6]:
vol = load_volume(scan_path)

In [7]:
interactive_visualize(vol)

interactive(children=(IntSlider(value=150, description='slice_number', max=299), Output()), _dom_classes=('wid…

In [8]:
vol2 = load_volume2(scan_path)

In [9]:
interactive_visualize(vol2)

interactive(children=(IntSlider(value=150, description='slice_number', max=299), Output()), _dom_classes=('wid…